In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [4]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [5]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.5157177
y: 0.5437437
z: -1.4220183
visibility: 0.9997907
, x: 0.5509759
y: 0.45666382
z: -1.3068162
visibility: 0.9996081
, x: 0.57318646
y: 0.45396054
z: -1.3069919
visibility: 0.99974877
, x: 0.5937662
y: 0.45290554
z: -1.3070966
visibility: 0.9995664
, x: 0.47642642
y: 0.4695021
z: -1.3389914
visibility: 0.9996008
, x: 0.4500509
y: 0.47576687
z: -1.3381928
visibility: 0.99975914
, x: 0.4268666
y: 0.48257923
z: -1.3389537
visibility: 0.9996375
, x: 0.6294322
y: 0.49800634
z: -0.6502274
visibility: 0.99966526
, x: 0.39904827
y: 0.5312983
z: -0.78237236
visibility: 0.9998221
, x: 0.5641879
y: 0.62808937
z: -1.1784129
visibility: 0.99971026
, x: 0.4781669
y: 0.6452566
z: -1.2174412
visibility: 0.9998136
, x: 0.802647
y: 0.97762924
z: -0.25315556
visibility: 0.992427
, x: 0.2324913
y: 0.99738824
z: -0.5911528
visibility: 0.9970155
, x: 0.9378767
y: 1.4932005
z: -0.42174253
visibility: 0.057181954
, x: 0.1249879
y: 1.5476627
z: -0.8781473
visibility: 0.23928323
, x: 0.9456499
y: 1.8

In [6]:
len(landmarks)

33

In [7]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [8]:
[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

[0.22204966843128204, 0.9956976175308228]

In [9]:
[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

[0.11821909248828888, 1.4856626987457275]

In [10]:
[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

[0.26326677203178406, 1.6425307989120483]

In [11]:
def calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist):
    right_shoulder = np.array(right_shoulder)  # Shoulder landmark
    right_elbow = np.array(right_elbow)  # Elbow landmark
    right_wrist = np.array(right_wrist)  # Wrist landmark
    
    # Calculate the angle between the vectors formed by the landmarks
    angle = np.degrees(np.arccos(np.dot(right_shoulder - right_elbow, right_wrist - right_elbow) / (np.linalg.norm(right_shoulder - right_elbow) * np.linalg.norm(right_wrist - right_elbow))))
    
    return angle


In [12]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


In [13]:
right_shoulder, right_elbow, right_wrist

([0.22204966843128204, 0.9956976175308228],
 [0.11821909248828888, 1.4856626987457275],
 [0.26326677203178406, 1.6425307989120483])

In [14]:
calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)

125.27728022591933

In [15]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(168, 788)

Start of Bench Press form logic


In [16]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.11821909
y: 1.4856627
z: -0.97380465
visibility: 0.15189455

In [17]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.22204967
y: 0.9956976
z: -0.38527143
visibility: 0.9918231

In [18]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.31407636
y: 2.004937
z: 0.08427195
visibility: 0.0008430867

In [19]:
def calculate_form_angle(elbow, shoulder, hip):
    elbow = np.array(elbow)  # Right elbow landmark
    shoulder = np.array(shoulder)  # Right shoulder landmark
    hip = np.array(hip)      # Right hip landmark
    
    
    radians = np.arctan2(shoulder[1]-hip[1], shoulder[0]-hip[0]) - np.arctan2(elbow[1]-hip[1], elbow[0]-hip[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle


In [20]:
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]


In [22]:
elbow, shoulder, hip

([0.11821909248828888, 1.4856626987457275],
 [0.22204966843128204, 0.9956976175308228],
 [0.31407636404037476, 2.004936933517456])

In [23]:
calculate_form_angle(elbow, shoulder, hip)

15.455159793864958

In [24]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(168, 788)

In [26]:
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate ohp angle
            overhead_angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            form_angle = calculate_form_angle(elbow, shoulder, hip)

            
            # Visualize angle
            cv2.putText(image, str(overhead_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(form_angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Overhead Press Checker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [28]:
cap = cv2.VideoCapture(0)

# OHP counter variables
counter = 0 
stage = None
feedback = ""

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            
            
            # Calculate ohp angle
            overhead_angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            form_angle = calculate_form_angle(elbow, shoulder, hip)
            
            # Visualize angle
            cv2.putText(image, str(overhead_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(form_angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Overhead Press Counter Algorithm
            if overhead_angle < 90:
                stage = "down"
            if overhead_angle > 150 and stage =="down":
                stage="up"
                counter +=1
            if form_angle > 25: 
                feedback = "Tuck elbows in, elbows are too tucked out"
            elif form_angle < 25:
                feedback = "Good form, keep it up"
                       
        except:
            pass
        
        # Render Overhead Press Counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
         # Text for 'SQUATS'
        cv2.putText(image, 'BENCH PRESS', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'STAGE'
        cv2.putText(image, 'STAGE', (165, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (160, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'FORM' (Ensure it fits within the box)
        feedback_lines = feedback.split('\n')
        if len(feedback_lines) > 0:
            feedback_line1 = feedback_lines[0]
        else:
            feedback_line1 = ''

        if len(feedback_lines) > 1:
            feedback_line2 = feedback_lines[1]
        else:
            feedback_line2 = ''

        cv2.putText(image, 'FORM', (290, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, feedback_line1, (270, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, feedback_line2, (270, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Bench Press Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    
    cv2.destroyAllWindows()
